<a href="https://colab.research.google.com/github/timwu64/Generative-AI---LoRA-FineTuning/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRa
* Model: bert-base-cased
* Evaluation approach: accuracy
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
# ! pip install -q "datasets==2.15.0"
# ! pip install scikit-learn
# ! pip install datasets
# ! pip install transformers
# ! pip install peft
# ! pip install evaluate

In [2]:
# Includes the relevant imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

SEED = 42

In [3]:
# Define the Foundation Model

base_model = "bert-base-cased"
peft_model_name = base_model+'-peft'
modified_base = base_model+'-modified'


# Tokenize the dataset with the foundation model
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load a pretrained Hugging Face model that can be used for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Includes the relevant imports and loads a Hugging Face dataset that can be used for sequence classification.
# Load and preprocess a dataset
from datasets import load_dataset

# Define the dataset
dataset_name = "imdb"

# Set dataset range
dataset_range = 1000

# Import the datasets and transformers packages
dataset = load_dataset(dataset_name)

# preprocess the dataset by returning tokenized examples
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Includes relevant imports and loads a Hugging Face tokenizer that can be used to prepare the dataset.
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# A subset of the full dataset to be used to reduce computational resources needed.
train_dataset = tokenized_datasets["train"].shuffle(seed=SEED).select(range(dataset_range))
eval_dataset = tokenized_datasets["test"].shuffle(seed=SEED).select(range(dataset_range))

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [6]:
from transformers import TrainingArguments, Trainer
# hyperparameters
lr = 1e-3
batch_size = 16
num_epochs = 8

# Define training args for base
training_args_base = TrainingArguments(
    output_dir='./foundation_results',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [7]:
trainer_base = Trainer(
    model=model,
    args=training_args_base,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [8]:
# Evaluate the pretrained model
# Accuracy classification metric is calculated using the dataset and pretrained model
base_results = trainer_base.evaluate()
print("Pre-trained Model:", base_results['eval_accuracy'])

Pre-trained Model: 0.516


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [9]:
from peft import LoraConfig, TaskType

# Initializes a Hugging Face PEST config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [10]:
# Includes the relveant imports
from peft import get_peft_model

# Creates a PEFT model using the config
peft_model = get_peft_model(model, lora_config)

In [11]:
# Define training args for peft
training_args_peft = TrainingArguments(
    output_dir='./prft_results',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


In [12]:
print('PEFT Model')
peft_model.print_trainable_parameters()

PEFT Model
trainable params: 38,402 || all params: 108,350,212 || trainable%: 0.03544247795288116


In [13]:
trainer_peft = Trainer(
    model=peft_model,
    args=training_args_peft,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
# Train the PRFT model

print('PEFT Model')
peft_model.print_trainable_parameters()
trainer_peft.train()

PEFT Model
trainable params: 38,402 || all params: 108,350,212 || trainable%: 0.03544247795288116


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.723889,0.583000
2,No log,0.389572,0.846000
3,No log,0.357224,0.864000
4,No log,0.361418,0.875000
5,No log,0.381158,0.860000
6,No log,0.333674,0.879000
7,No log,0.369139,0.877000
8,0.328400,0.361570,0.877000


TrainOutput(global_step=504, training_loss=0.3264605936313432, metrics={'train_runtime': 846.4772, 'train_samples_per_second': 9.451, 'train_steps_per_second': 0.595, 'total_flos': 2105832210432000.0, 'train_loss': 0.3264605936313432, 'epoch': 8.0})

In [15]:
peft_results = trainer_peft.evaluate()
print("Compare the result for Pre-trained and PEFT Model:")
print("Pre-trained Model:", base_results['eval_accuracy'])
print("PEFT Model:", peft_results['eval_accuracy'])

Compare the result for Pre-trained and PEFT Model:
Pre-trained Model: 0.516
PEFT Model: 0.879


In [16]:
# Save the PEFT model, Fine-tuened parameters are saved to a separate directory. The saved weights directory should be the same home directory as the notebook file

# Save our LoRA model & tokenizer results
peft_model.save_pretrained(peft_model_name)
tokenizer.save_pretrained(modified_base)

('bert-base-cased-modified/tokenizer_config.json',
 'bert-base-cased-modified/special_tokens_map.json',
 'bert-base-cased-modified/vocab.txt',
 'bert-base-cased-modified/added_tokens.json',
 'bert-base-cased-modified/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [17]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
config = PeftConfig.from_pretrained(peft_model_name)

# LOAD the Saved PEFT model
tokenizer = AutoTokenizer.from_pretrained(modified_base)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_name, device_map={"":0})
model.eval()

print("Peft model loaded")


Peft model loaded


In [18]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer_loaded_peft = Trainer(
    model=model,
    args=training_args_base,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Evaluate the Peft model
# Accuracy classification metric is calculated using the dataset and pretrained model
loaded_peft_results = trainer_loaded_peft.evaluate()
print("Pre-trained Model:", base_results['eval_accuracy'])
print("Loaded PEFT Model:", loaded_peft_results['eval_accuracy'])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Pre-trained Model: 0.516
Loaded PEFT Model: 0.879
